In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [ ]:
test_data_orig = pd.read_csv(os.path.join('data', 'loanDataTest.csv'))
pd.set_option('display.max_columns', None)
test_data_orig

In [ ]:
#train_data_orig = pd.read_csv(os.path.join('data', 'loanDataTrain.csv'))
pd.set_option('display.max_columns', None)
train_data_orig

#test_data_orig = pd.read_csv(os.path.join('data', 'loanDataTest.csv'))
pd.set_option('display.max_columns', None)
test_data_orig

#train_data_orig = pd.read_csv(os.path.join('data', 'loanDataTrain.csv'))
pd.set_option('display.max_columns', None)
train_data_orig

#Check values in test and train sets
test_Gender = test_data_orig.Gender.unique()
train_Gender = train_data_orig.Gender.unique()
test_Dependents = test_data_orig.Dependents.unique()
train_Dependents = train_data_orig.Dependents.unique()
test_Education = test_data_orig.Education.unique()
train_Education = train_data_orig.Education.unique()
test_Self_Employed = test_data_orig.Self_Employed.unique()
train_Self_Employed = train_data_orig.Self_Employed.unique()
test_ApplicantIncome = test_data_orig.ApplicantIncome.unique()
train_ApplicantIncome = train_data_orig.ApplicantIncome.unique()
test_CoapplicantIncome = test_data_orig.CoapplicantIncome.unique()
train_CoapplicantIncome = train_data_orig.CoapplicantIncome.unique()
test_LoanAmount = test_data_orig.LoanAmount.unique()
train_LoanAmount = train_data_orig.LoanAmount.unique()
test_Loan_Amount_Term = test_data_orig.Loan_Amount_Term.unique()
train_Loan_Amount_Term = train_data_orig.Loan_Amount_Term.unique()
test_Credit_History = test_data_orig.Credit_History.unique()
train_Credit_History = train_data_orig.Credit_History.unique()
test_Property_Area = test_data_orig.Property_Area.unique()
train_Property_Area = train_data_orig.Property_Area.unique()

train_Loan_Status = train_data_orig.Loan_Status.unique()

#Print out values
#test_Gender, train_Gender, test_Dependents, train_Dependents, test_Education, train_Education, test_Self_Employed, train_Self_Employed, test_ApplicantIncome, train_ApplicantIncome, test_CoapplicantIncome, train_CoapplicantIncome, test_LoanAmount, train_LoanAmount, test_Loan_Amount_Term, train_Loan_Amount_Term, test_Credit_History, train_Credit_History, test_Property_Area, train_Property_Area, train_Loan_Status

#drop nans
test_data = test_data_orig.dropna(axis=0)
train_data_pre_dummy = train_data_orig.dropna(axis=0)

test_Gender = test_data.Gender.unique()
train_Gender = train_data_pre_dummy.Gender.unique()
test_Dependents = test_data.Dependents.unique()
train_Dependents = train_data_pre_dummy.Dependents.unique()
test_Education = test_data.Education.unique()
train_Education = train_data_pre_dummy.Education.unique()
test_Self_Employed = test_data.Self_Employed.unique()
train_Self_Employed = train_data_pre_dummy.Self_Employed.unique()
test_ApplicantIncome = test_data.ApplicantIncome.unique()
train_ApplicantIncome = train_data_pre_dummy.ApplicantIncome.unique()
test_CoapplicantIncome = test_data.CoapplicantIncome.unique()
train_CoapplicantIncome = train_data_pre_dummy.CoapplicantIncome.unique()
test_LoanAmount = test_data.LoanAmount.unique()
train_LoanAmount = train_data_pre_dummy.LoanAmount.unique()
test_Loan_Amount_Term = test_data.Loan_Amount_Term.unique()
train_Loan_Amount_Term = train_data_pre_dummy.Loan_Amount_Term.unique()
test_Credit_History = test_data.Credit_History.unique()
train_Credit_History = train_data_pre_dummy.Credit_History.unique()
test_Property_Area = test_data.Property_Area.unique()
train_Property_Area = train_data_pre_dummy.Property_Area.unique()

train_Loan_Status = train_data_pre_dummy.Loan_Status.unique()

#Print out categorical values
test_Gender, train_Gender, test_Dependents, train_Dependents, test_Education, train_Education, test_Self_Employed, train_Self_Employed, train_Loan_Amount_Term, test_Credit_History, train_Credit_History, test_Property_Area, train_Property_Area, train_Loan_Status

#Make loan status a dummy variable, drop the "N" column
train_data = pd.get_dummies(train_data_pre_dummy, prefix=['Loan_Status'], columns=['Loan_Status'])
train_data.drop('Loan_Status_N', axis=1, inplace=True)
train_data

#Replace "3+" in dependents with "3" - not perfect, but easier to deal with
train_data = train_data.replace(['3+'],'3')
test_data = test_data.replace(['3+'],'3')

#Limited dummy variables
train_data_limit_dummies = pd.get_dummies(train_data, prefix=['Gender'], columns=['Gender'])
train_data_limit_dummies.drop('Gender_Male', axis=1, inplace=True)
test_data_limit_dummies = pd.get_dummies(test_data, prefix=['Gender'], columns=['Gender'])
test_data_limit_dummies.drop('Gender_Male', axis=1, inplace=True)
train_data_limit_dummies = pd.get_dummies(train_data_limit_dummies, prefix=['Education'], columns=['Education'])
train_data_limit_dummies.drop('Education_Not Graduate', axis=1, inplace=True)
test_data_limit_dummies = pd.get_dummies(test_data_limit_dummies, prefix=['Education'], columns=['Education'])
test_data_limit_dummies.drop('Education_Not Graduate', axis=1, inplace=True)
train_data_limit_dummies = pd.get_dummies(train_data_limit_dummies, prefix=['Married'], columns=['Married'])
train_data_limit_dummies.drop('Married_No', axis=1, inplace=True)
test_data_limit_dummies = pd.get_dummies(test_data_limit_dummies, prefix=['Married'], columns=['Married'])
test_data_limit_dummies.drop('Married_No', axis=1, inplace=True)
train_data_limit_dummies = pd.get_dummies(train_data_limit_dummies, prefix=['Self_Employed'], columns=['Self_Employed'])
train_data_limit_dummies.drop('Self_Employed_No', axis=1, inplace=True)
test_data_limit_dummies = pd.get_dummies(test_data_limit_dummies, prefix=['Self_Employed'], columns=['Self_Employed'])
test_data_limit_dummies.drop('Self_Employed_No', axis=1, inplace=True)
train_data_limit_dummies = pd.get_dummies(train_data_limit_dummies, prefix=['Credit_History'], columns=['Credit_History'])
train_data_limit_dummies.drop('Credit_History_0.0', axis=1, inplace=True)
test_data_limit_dummies = pd.get_dummies(test_data_limit_dummies, prefix=['Credit_History'], columns=['Credit_History'])
test_data_limit_dummies.drop('Credit_History_0.0', axis=1, inplace=True)

train_data_limit_dummies = pd.get_dummies(train_data_limit_dummies, prefix=['Property_Area'], columns=['Property_Area'])
test_data_limit_dummies = pd.get_dummies(test_data_limit_dummies, prefix=['Property_Area'], columns=['Property_Area'])

train_data_limit_dummies

## Grid Search

### 1 (less_dummies) param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01]}, random state 33, all columns - some improvement on more_dummies version

In [ ]:
target = train_data_limit_dummies["Loan_Status_Y"]
target_names = ["Denied", "Approved"]
data = train_data_limit_dummies.drop(["Loan_Status_Y", "Loan_ID"], axis=1)
feature_names = data.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=33)
model = SVC(kernel='linear', cache_size=1000)

In [ ]:
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3, n_jobs=-1)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)

In [ ]:
predictions = grid.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=["Denied", "Approved"]))

Run 1 (limit dummies) summary

GridSearchCV(estimator=SVC(cache_size=1000, kernel='linear'), n_jobs=-1,
             param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01]},
             verbose=3)
             
Best parameters: {'C': 5, 'gamma': 0.0001}

              precision    recall  f1-score   support

      Denied       0.87      0.33      0.48        39
    Approved       0.75      0.98      0.85        81

    accuracy                           0.77       120
   macro avg       0.81      0.65      0.67       120
weighted avg       0.79      0.77      0.73       120

### 2 (less dummies). param_grid={'C': [1, 5, 10], 'gamma': [0.00001, 0.0001, 0.001]}, random state 33, all columns - no discernable change from #1

In [ ]:
target = train_data_limit_dummies["Loan_Status_Y"]
target_names = ["Denied", "Approved"]
data = train_data_limit_dummies.drop(["Loan_Status_Y", "Loan_ID"], axis=1)
feature_names = data.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=33)
model = SVC(kernel='linear', cache_size=1000)

In [ ]:
param_grid = {'C': [1, 5, 10],
              'gamma': [0.00001, 0.0001, 0.001]}
grid = GridSearchCV(model, param_grid, verbose=3, n_jobs=-1)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)

In [ ]:
predictions = grid.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=["Denied", "Approved"]))

Run 2 (limit dummies) summary

GridSearchCV(estimator=SVC(cache_size=1000, kernel='linear'), n_jobs=-1,
             param_grid={'C': [1, 5, 10], 'gamma': [1e-05, 0.0001, 0.001]},
             verbose=3)
Best parameters: {'C': 5, 'gamma': 1e-05}

              precision    recall  f1-score   support

      Denied       0.87      0.33      0.48        39
    Approved       0.75      0.98      0.85        81

    accuracy                           0.77       120
   macro avg       0.81      0.65      0.67       120
weighted avg       0.79      0.77      0.73       120

### 3 (less dummies). From https://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html#sphx-glr-auto-examples-model-selection-plot-nested-cross-validation-iris-py: nested vs. non-nested cross-validation -  param_grid={'C': [1, 5, 10], 'gamma': [0.00001, 0.0001, 0.001]} . Non-nested slightly higher, scores a little higher than with data_more_dummies

In [ ]:
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
import numpy as np

In [ ]:
NUM_TRIALS = 30

In [ ]:
target = train_data_limit_dummies["Loan_Status_Y"]
target_names = ["Denied", "Approved"]

In [ ]:
data = train_data_limit_dummies.drop(["Loan_Status_Y", "Loan_ID"], axis=1)
feature_names = data.columns

In [ ]:
p_grid = {"C": [1, 5, 10],
          "gamma": [0.00001, 0.0001, 0.001]}

In [ ]:
svm = SVC(kernel="rbf", cache_size=1000)

In [ ]:
non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

In [ ]:
# Loop for each trial
for i in range(NUM_TRIALS):

    # Choose cross-validation techniques for the inner and outer loops,
    # independently of the dataset.
    # E.g "GroupKFold", "LeaveOneOut", "LeaveOneGroupOut", etc.
    inner_cv = KFold(n_splits=4, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=4, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=svm, param_grid=p_grid, cv=inner_cv, verbose=2, n_jobs=-1)
    clf.fit(data, target)
    non_nested_scores[i] = clf.best_score_

    # Nested CV with parameter optimization
    nested_score = cross_val_score(clf, X=data, y=target, cv=outer_cv, verbose=2, n_jobs=-1)
    nested_scores[i] = nested_score.mean()

In [ ]:
score_difference = non_nested_scores - nested_scores

In [ ]:
print("Average difference of {:6f} with std. dev. of {:6f}."
      .format(score_difference.mean(), score_difference.std()))

In [ ]:
# Plot scores on each trial for nested and non-nested CV
plt.figure()
plt.subplot(211)
non_nested_scores_line, = plt.plot(non_nested_scores, color='r')
nested_line, = plt.plot(nested_scores, color='b')
plt.ylabel("score", fontsize="14")
plt.legend([non_nested_scores_line, nested_line],
           ["Non-Nested CV", "Nested CV"],
           bbox_to_anchor=(0, .4, .5, 0))
plt.title("Non-Nested and Nested Cross Validation on train_data_limit_dummies",
          x=.5, y=1.1, fontsize="15")

In [ ]:
# Plot bar chart of the difference.
plt.subplot(212)
difference_plot = plt.bar(range(NUM_TRIALS), score_difference)
plt.xlabel("Individual Trial #")
plt.legend([difference_plot],
           ["Non-Nested CV - Nested CV Score"],
           bbox_to_anchor=(0, 1, .8, 0))
plt.ylabel("score difference", fontsize="14")

plt.show()

Run 3 summary

NUM_TRIALS = 30
Average difference of 0.006944 with std. dev. of 0.006896.

### 4 (less dummies). Nested vs. non-nested - param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01]}. Non-nested slightly higher, scores a little higher than with data_more_dummies

In [ ]:
NUM_TRIALS = 30

In [ ]:
target = train_data_limit_dummies["Loan_Status_Y"]
target_names = ["Denied", "Approved"]
data = train_data_limit_dummies.drop(["Loan_Status_Y", "Loan_ID"], axis=1)
feature_names = data.columns

In [ ]:
p_grid = {"C": [1, 5, 10],
          "gamma": [0.0001, 0.001, 0.01]}

In [ ]:
svm = SVC(kernel="rbf", cache_size=1000)

In [ ]:
non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

In [ ]:
# Loop for each trial
for i in range(NUM_TRIALS):

    # Choose cross-validation techniques for the inner and outer loops,
    # independently of the dataset.
    # E.g "GroupKFold", "LeaveOneOut", "LeaveOneGroupOut", etc.
    inner_cv = KFold(n_splits=4, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=4, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=svm, param_grid=p_grid, cv=inner_cv, verbose=2, n_jobs=-1)
    clf.fit(data, target)
    non_nested_scores[i] = clf.best_score_

    # Nested CV with parameter optimization
    nested_score = cross_val_score(clf, X=data, y=target, cv=outer_cv, verbose=2, n_jobs=-1)
    nested_scores[i] = nested_score.mean()

In [ ]:
score_difference = non_nested_scores - nested_scores

In [ ]:
print("Average difference of {:6f} with std. dev. of {:6f}."
      .format(score_difference.mean(), score_difference.std()))

In [ ]:
# Plot scores on each trial for nested and non-nested CV
plt.figure()
plt.subplot(211)
non_nested_scores_line, = plt.plot(non_nested_scores, color='r')
nested_line, = plt.plot(nested_scores, color='b')
plt.ylabel("score", fontsize="14")
plt.legend([non_nested_scores_line, nested_line],
           ["Non-Nested CV", "Nested CV"],
           bbox_to_anchor=(0, .4, .5, 0))
plt.title("Non-Nested and Nested Cross Validation on train_data_limit_dummies",
          x=.5, y=1.1, fontsize="15")

In [ ]:
# Plot bar chart of the difference.
plt.subplot(212)
difference_plot = plt.bar(range(NUM_TRIALS), score_difference)
plt.xlabel("Individual Trial #")
plt.legend([difference_plot],
           ["Non-Nested CV - Nested CV Score"],
           bbox_to_anchor=(0, 1, .8, 0))
plt.ylabel("score difference", fontsize="14")

plt.show()

Run 4 summary

NUM_TRIALS = 30
Average difference of 0.008819 with std. dev. of 0.004862.

### 5 (limit_dummies). Nested vs. non-nested - param_grid={'C': [1, 5, 10], 'gamma':  [0.00001, 0.0001, 0.001]}: As in #3, but with SVC(kernel="linear") rather than rbf -

In [ ]:
# Data and parameters
NUM_TRIALS = 30
target = train_data_limit_dummies["Loan_Status_Y"]
target_names = ["Denied", "Approved"]
data = train_data_limit_dummies.drop(["Loan_Status_Y", "Loan_ID"], axis=1)
feature_names = data.columns
p_grid = {"C": [1, 5, 10],
          "gamma": [0.00001, 0.0001, 0.001]}
svm = SVC(kernel="linear", cache_size=1000)
non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

In [ ]:
# Loop for each trial
for i in range(NUM_TRIALS):

    # Choose cross-validation techniques for the inner and outer loops,
    # independently of the dataset.
    # E.g "GroupKFold", "LeaveOneOut", "LeaveOneGroupOut", etc.
    inner_cv = KFold(n_splits=4, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=4, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=svm, param_grid=p_grid, cv=inner_cv, verbose=2, n_jobs=-1)
    clf.fit(data, target)
    non_nested_scores[i] = clf.best_score_

    # Nested CV with parameter optimization
    nested_score = cross_val_score(clf, X=data, y=target, cv=outer_cv, verbose=2, n_jobs=-1)
    nested_scores[i] = nested_score.mean()

In [ ]:
score_difference = non_nested_scores - nested_scores
print("Average difference of {:6f} with std. dev. of {:6f}."
      .format(score_difference.mean(), score_difference.std()))

In [ ]:
# Plot scores on each trial for nested and non-nested CV
plt.figure()
plt.subplot(211)
non_nested_scores_line, = plt.plot(non_nested_scores, color='r')
nested_line, = plt.plot(nested_scores, color='b')
plt.ylabel("score", fontsize="14")
plt.legend([non_nested_scores_line, nested_line],
           ["Non-Nested CV", "Nested CV"],
           bbox_to_anchor=(0, .4, .5, 0))
plt.title("Non-Nested and Nested Cross Validation on train_data_limit_dummies",
          x=.5, y=1.1, fontsize="15")

In [ ]:
# Plot bar chart of the difference.
plt.subplot(212)
difference_plot = plt.bar(range(NUM_TRIALS), score_difference)
plt.xlabel("Individual Trial #")
plt.legend([difference_plot],
           ["Non-Nested CV - Nested CV Score"],
           bbox_to_anchor=(0, 1, .8, 0))
plt.ylabel("score difference", fontsize="14")

plt.show()

In [ ]:
print(grid.best_params_)

Run 5 summary

### 6 (limited dummies). As #4 but with scaling and larger cache size. Nested vs. non-nested - param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01]}, kernel = rbf: No nested/non-nested difference

https://scikit-learn.org/stable/modules/svm.html#svm-classification Kernel cache size: For SVC, SVR, NuSVC and NuSVR, the size of the kernel cache has a strong impact on run times for larger problems. If you have enough RAM available, it is recommended to set cache_size to a higher value than the default of 200(MB), such as 500(MB) or 1000(MB). 

With 64 GB, I'd say I could go higher on the next run.

Support Vector Machine algorithms are not scale invariant, so it is highly recommended to scale your data. REDO AT LEAST FOR MODELS THAT RUN In REASONABLE TIME

In [ ]:
NUM_TRIALS = 30

In [ ]:
target = train_data_limit_dummies["Loan_Status_Y"]
target_names = ["Denied", "Approved"]
data = train_data_limit_dummies.drop(["Loan_Status_Y", "Loan_ID"], axis=1)
feature_names = data.columns

In [ ]:
data_scaler = MinMaxScaler().fit(data)
data_scaled = data_scaler.transform(data)

In [ ]:
p_grid = {"C": [1, 5, 10],
          "gamma": [0.0001, 0.001, 0.01]}

In [ ]:
svm = SVC(kernel="rbf", cache_size=1000)

In [ ]:
non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

In [ ]:
# Loop for each trial
for i in range(NUM_TRIALS):

    # Choose cross-validation techniques for the inner and outer loops,
    # independently of the dataset.
    # E.g "GroupKFold", "LeaveOneOut", "LeaveOneGroupOut", etc.
    inner_cv = KFold(n_splits=4, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=4, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=svm, param_grid=p_grid, cv=inner_cv, verbose=2, n_jobs=-1)
    clf.fit(data_scaled, target)
    non_nested_scores[i] = clf.best_score_

    # Nested CV with parameter optimization
    nested_score = cross_val_score(clf, X=data_scaled, y=target, cv=outer_cv, verbose=2, n_jobs=-1)
    nested_scores[i] = nested_score.mean()

In [ ]:
score_difference = non_nested_scores - nested_scores

In [ ]:
print("Average difference of {:6f} with std. dev. of {:6f}."
      .format(score_difference.mean(), score_difference.std()))

In [ ]:
# Plot scores on each trial for nested and non-nested CV
plt.figure()
plt.subplot(211)
non_nested_scores_line, = plt.plot(non_nested_scores, color='r')
nested_line, = plt.plot(nested_scores, color='b')
plt.ylabel("score", fontsize="14")
plt.legend([non_nested_scores_line, nested_line],
           ["Non-Nested CV", "Nested CV"],
           bbox_to_anchor=(0, .4, .5, 0))
plt.title("Non-Nested and Nested Cross Validation on train_data_limit_dummies",
          x=.5, y=1.1, fontsize="15")

In [ ]:
# Plot bar chart of the difference.
plt.subplot(212)
difference_plot = plt.bar(range(NUM_TRIALS), score_difference)
plt.xlabel("Individual Trial #")
plt.legend([difference_plot],
           ["Non-Nested CV - Nested CV Score"],
           bbox_to_anchor=(0, 1, .8, 0))
plt.ylabel("score difference", fontsize="14")

plt.show()

#Run 6 summary
NUM_TRIALS = 30
Average difference of 0.000208 with std. dev. of 0.001122.

### 7 (limited dummies). As #6 (limited dummies) but kernel="linear" 

In [ ]:
NUM_TRIALS = 30

In [ ]:
target = train_data_limit_dummies["Loan_Status_Y"]
target_names = ["Denied", "Approved"]
data = train_data_limit_dummies.drop(["Loan_Status_Y", "Loan_ID"], axis=1)
feature_names = data.columns

In [ ]:
data_scaler = MinMaxScaler().fit(data)
data_scaled = data_scaler.transform(data)

In [ ]:
p_grid = {"C": [1, 5, 10],
          "gamma": [0.0001, 0.001, 0.01]}

In [ ]:
svm = SVC(kernel="linear", cache_size=1000)

In [ ]:
non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

In [ ]:
# Loop for each trial
for i in range(NUM_TRIALS):

    # Choose cross-validation techniques for the inner and outer loops,
    # independently of the dataset.
    # E.g "GroupKFold", "LeaveOneOut", "LeaveOneGroupOut", etc.
    inner_cv = KFold(n_splits=4, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=4, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=svm, param_grid=p_grid, cv=inner_cv, verbose=2, n_jobs=-1)
    clf.fit(data_scaled, target)
    non_nested_scores[i] = clf.best_score_

    # Nested CV with parameter optimization
    nested_score = cross_val_score(clf, X=data_scaled, y=target, cv=outer_cv, verbose=2, n_jobs=-1)
    nested_scores[i] = nested_score.mean()

In [ ]:
score_difference = non_nested_scores - nested_scores

In [ ]:
print("Average difference of {:6f} with std. dev. of {:6f}."
      .format(score_difference.mean(), score_difference.std()))

In [ ]:
# Plot scores on each trial for nested and non-nested CV
plt.figure()
plt.subplot(211)
non_nested_scores_line, = plt.plot(non_nested_scores, color='r')
nested_line, = plt.plot(nested_scores, color='b')
plt.ylabel("score", fontsize="14")
plt.legend([non_nested_scores_line, nested_line],
           ["Non-Nested CV", "Nested CV"],
           bbox_to_anchor=(0, .4, .5, 0))
plt.title("Non-Nested and Nested Cross Validation on train_data_limit_dummies",
          x=.5, y=1.1, fontsize="15")

In [ ]:
# Plot bar chart of the difference.
plt.subplot(212)
difference_plot = plt.bar(range(NUM_TRIALS), score_difference)
plt.xlabel("Individual Trial #")
plt.legend([difference_plot],
           ["Non-Nested CV - Nested CV Score"],
           bbox_to_anchor=(0, 1, .8, 0))
plt.ylabel("score difference", fontsize="14")

plt.show()

#Run 7 summary
NUM_TRIALS = 30
Average difference of 0.000000 with std. dev. of 0.000000.
Guess nested vs. not doesn't matter for linear as opposed to rbf, or anyway not here?

### 8 (limited dummies). As #7 (limited dummies) but starting with data edits: take out Loan_Amount_Term column. Scores overall a bit lower than in 7. No nested/non-nested difference

In [ ]:
NUM_TRIALS = 30

In [ ]:
target = train_data_limit_dummies["Loan_Status_Y"]
target_names = ["Denied", "Approved"]
data = train_data_limit_dummies.drop(["Loan_Status_Y", "Loan_ID", 'Loan_Amount_Term'], axis=1)
feature_names = data.columns

In [ ]:
data_scaler = MinMaxScaler().fit(data)
data_scaled = data_scaler.transform(data)

In [ ]:
p_grid = {"C": [1, 5, 10],
          "gamma": [0.0001, 0.001, 0.01]}

In [ ]:
svm = SVC(kernel="linear", cache_size=1000)

In [ ]:
non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

In [ ]:
# Loop for each trial
for i in range(NUM_TRIALS):

    # Choose cross-validation techniques for the inner and outer loops,
    # independently of the dataset.
    # E.g "GroupKFold", "LeaveOneOut", "LeaveOneGroupOut", etc.
    inner_cv = KFold(n_splits=4, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=4, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=svm, param_grid=p_grid, cv=inner_cv, verbose=2, n_jobs=-1)
    clf.fit(data_scaled, target)
    non_nested_scores[i] = clf.best_score_

    # Nested CV with parameter optimization
    nested_score = cross_val_score(clf, X=data_scaled, y=target, cv=outer_cv, verbose=2, n_jobs=-1)
    nested_scores[i] = nested_score.mean()

In [ ]:
score_difference = non_nested_scores - nested_scores

In [ ]:
print("Average difference of {:6f} with std. dev. of {:6f}."
      .format(score_difference.mean(), score_difference.std()))

In [ ]:
# Plot scores on each trial for nested and non-nested CV
plt.figure()
plt.subplot(211)
non_nested_scores_line, = plt.plot(non_nested_scores, color='r')
nested_line, = plt.plot(nested_scores, color='b')
plt.ylabel("score", fontsize="14")
plt.legend([non_nested_scores_line, nested_line],
           ["Non-Nested CV", "Nested CV"],
           bbox_to_anchor=(0, .4, .5, 0))
plt.title("Non-Nested and Nested Cross Validation on train_data_limit_dummies",
          x=.5, y=1.1, fontsize="15")

In [ ]:
# Plot bar chart of the difference.
plt.subplot(212)
difference_plot = plt.bar(range(NUM_TRIALS), score_difference)
plt.xlabel("Individual Trial #")
plt.legend([difference_plot],
           ["Non-Nested CV - Nested CV Score"],
           bbox_to_anchor=(0, 1, .8, 0))
plt.ylabel("score difference", fontsize="14")

plt.show()

#Run 8 summary: Scores overall are a bit lower than in 7. Where nested/non-nested differ, non-nested is slightly higher.
NUM_TRIALS = 30
Average difference of 0.000625 with std. dev. of 0.001096.

### 9 (limited dummies). As #8 (limited dummies) but further data edits: take out Loan_Amount_Term and LoanAmount columns, since these could be considered results. No nested/non-nested difference.

In [ ]:
NUM_TRIALS = 30

In [ ]:
target = train_data_limit_dummies["Loan_Status_Y"]
target_names = ["Denied", "Approved"]
data = train_data_limit_dummies.drop(["Loan_Status_Y", "Loan_ID", 'LoanAmount', 'Loan_Amount_Term'], axis=1)
feature_names = data.columns

In [ ]:
data_scaler = MinMaxScaler().fit(data)
data_scaled = data_scaler.transform(data)

In [ ]:
p_grid = {"C": [1, 5, 10],
          "gamma": [0.0001, 0.001, 0.01]}

In [ ]:
svm = SVC(kernel="linear", cache_size=1000)

In [ ]:
non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

In [ ]:
# Loop for each trial
for i in range(NUM_TRIALS):

    # Choose cross-validation techniques for the inner and outer loops,
    # independently of the dataset.
    # E.g "GroupKFold", "LeaveOneOut", "LeaveOneGroupOut", etc.
    inner_cv = KFold(n_splits=4, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=4, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=svm, param_grid=p_grid, cv=inner_cv, verbose=2, n_jobs=-1)
    clf.fit(data_scaled, target)
    non_nested_scores[i] = clf.best_score_

    # Nested CV with parameter optimization
    nested_score = cross_val_score(clf, X=data_scaled, y=target, cv=outer_cv, verbose=2, n_jobs=-1)
    nested_scores[i] = nested_score.mean()

In [ ]:
score_difference = non_nested_scores - nested_scores

In [ ]:
print("Average difference of {:6f} with std. dev. of {:6f}."
      .format(score_difference.mean(), score_difference.std()))

In [ ]:
# Plot scores on each trial for nested and non-nested CV
plt.figure()
plt.subplot(211)
non_nested_scores_line, = plt.plot(non_nested_scores, color='r')
nested_line, = plt.plot(nested_scores, color='b')
plt.ylabel("score", fontsize="14")
plt.legend([non_nested_scores_line, nested_line],
           ["Non-Nested CV", "Nested CV"],
           bbox_to_anchor=(0, .4, .5, 0))
plt.title("Non-Nested and Nested Cross Validation on train_data_limit_dummies",
          x=.5, y=1.1, fontsize="15")

In [ ]:
# Plot bar chart of the difference.
plt.subplot(212)
difference_plot = plt.bar(range(NUM_TRIALS), score_difference)
plt.xlabel("Individual Trial #")
plt.legend([difference_plot],
           ["Non-Nested CV - Nested CV Score"],
           bbox_to_anchor=(0, 1, .8, 0))
plt.ylabel("score difference", fontsize="14")

plt.show()

### Run 9 summary: 
NUM_TRIALS = 30
Scores overall a tiny bit lower than for models with more columns, no difference between nested and non-nested